In [15]:
from dotenv import load_dotenv
from datasets import load_dataset
from openai import OpenAI
import pandas as pd
import json

In [5]:
load_dotenv()
client = OpenAI()

In [6]:
dataset = load_dataset('stanfordnlp/imdb')

In [9]:
# Let's convert them to dataframes
df_train = pd.DataFrame(dataset['train'])
df_test = pd.DataFrame(dataset['test'])

In [10]:
df_test.head(10)

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
5,I had high hopes for this one until they chang...,0
6,Isaac Florentine has made some of the best wes...,0
7,"It actually pains me to say it, but this movie...",0
8,"Technically I'am a Van Damme Fan, or I was. th...",0
9,"Honestly awful film, bad editing, awful lighti...",0


In [30]:
reviews = '\n'.join(['"""' + review + '"""' for review in df_test.loc[0:10].text])

In [34]:
reviews.split('\n')

['"""I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as they 

In [27]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_sentiment_of_texts",
            "description": "get sentiments of a list of texts",
            "parameters": {
                "type": "object",
                "properties": {
                    "sentiments_list": {
                        "type": "array",
                        "description": "a list of sentiments derived from a list of texts",
                        "items": {
                            "type": "object",
                            "description": "the sentiment of a single text from the list of texts",
                            "properties": {
                                "sentiment": {
                                    "type": "string",
                                    "enum": [
                                        "positive",
                                        "negative",
                                    ],
                                    "description": "The sentiment of the text, positive or negative"
                                }
                            },
                            "required": [
                                "sentiment"
                            ]
                        }
                    }
                }
            }
        }
    }
]

In [37]:
completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[
        {
            'role': 'system', 
            'content': 'You are an agent that takes in multiple sentences separated by triple quotes and newlines, and returns the sentiment, either positive or negative, of each of them. Ignore the HTML tags and escape sequences and just consider the text.'
        },
        {
            'role': 'user', 
            'content': f'{reviews}'
        }
    ],
    tools=tools,
    tool_choice="auto"
)

In [38]:
json.loads(completion.choices[0].message.tool_calls[0].function.arguments)

{'sentiments_list': [{'sentiment': 'positive'},
  {'sentiment': 'positive'},
  {'sentiment': 'negative'},
  {'sentiment': 'negative'},
  {'sentiment': 'positive'},
  {'sentiment': 'negative'},
  {'sentiment': 'negative'},
  {'sentiment': 'positive'},
  {'sentiment': 'negative'},
  {'sentiment': 'negative'}]}

In [18]:
sentiments = json.loads(completion.choices[0].message.content)['sentiments']

In [20]:
sentiment_nums = []
for sentiment in sentiments:
    if sentiment == 'negative':
        sentiment_nums.append(0)
    else:
        sentiment_nums.append(1)

In [21]:
sentiment_nums

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1]